## Spark Streaming 을 위한 데이터 준비

로컬 Docker 환경 내 Kafka Broker 와 Zookeeper 를 가정합니다.

https://github.com/1ambda/practical-data-pipeline-code 레포지토리 루트에서 아래 커맨드를 이용할 수 있습니다.

```bash
make compose.clean compose.storage-all
```

만약 Kafka Python Library 가 깔려있지 않을 경우 아래와 같이 Pyenv 환경 내에서 설치할 수 있습니다.

```bash
pyenv activate {PYENV_NAME} # 본인이 생성한 Pyenv 이름 설정
pip install kafka-python==2.0.2
pip install msgpack==1.0.3
```

In [1]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.client_async import KafkaClient
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import TopicPartition

import json
import msgpack

In [2]:
zookeeper_servers = ["127.0.0.1:2181"]
bootstrap_servers = ["127.0.0.1:9092"]

### Topic 생성

Kafka Partition 을 생성합니다. 일반적으로는 Kafka 관리자가 요청을 받아 UI 나 Console 에서 직접 생성할 수 있습니다.  
API 를 이용해 생성하는 경우는 많지 않으나, 이 노트북에선 실습을 위해 API 를 통해 생성하는 예제를 사용합니다.

In [3]:
admin = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

In [12]:
# partition = 3, replication = 1 설정을 가진 토픽을 생성합니다.
topic = "user-event"
admin.create_topics([NewTopic(topic, 3, 1)])

In [5]:
admin.list_topics()

['user-event', '_schemas', '__consumer_offsets']

### Kafka Producer 를 이용해 데이터 전송

kafka-python API 를 이용해 데이터를 Broker 로 전송합니다.  
일반적으로는 Java / Scala 언어로 이루어진 개발한 Producer 나 Kafka Connect 등을 이용합니다.  

다만 이 노트북에선 실습으로 Python API 를 이용하기 위해 Pandas 로 CSV 파일을 읽어 Python Kafka API 로 전송합니다.

In [6]:
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [7]:
DATASET_ROOT = "/Users/hoon.park/github/1ambda/practical-data-pipeline-code"
DATASET_AIRBNB_LISTING = f"{DATASET_ROOT}/_datasets/ecommerce/e-commerce.csv"

In [8]:
import pandas as pd
pdf = pd.read_csv(DATASET_AIRBNB_LISTING)

In [9]:
pdf.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,595414620,4adb70bb-edbd-4981-b60f-a05bfd32683a
1,2020-01-01 00:00:09 UTC,view,5812943,1487580012121948301,NaN,kinetics,3.97,595414640,c8c5205d-be43-4f1d-aa56-4828b8151c8a


In [14]:
# Pandas Row 는 Tuple 입니다. 이를 Dict 로 바꾸기 위해 to_dict 함수를 호출해
# 전달 받은 Row (Dict) 를 Kafka Producer 를 이용해 Topic 으로 보냅니다.

for row in pdf.head(50000).to_dict(orient="records"):
    # producer.send(topicAirbnbListing, bytes(str(row), 'utf-8'))
    del row['category_id']
    del row['category_code']
    del row['brand']
    del row['user_session']
    message = json.dumps(row).encode('utf-8')

    producer.send(topic, message)

### Console Consumer 로 데이터 읽기


이제 Broker 에 접근해 Console Consumer 로 해당 토픽 데이터를 읽어봅니다. 컨테이너에 들어가 다음 커맨드를 실행할 수 있습니다.  

```
# 터미널에서 도커로 로그인하기 위해 `docker exec -it broker /bin/bash`

kafka-console-consumer --bootstrap-server broker:9092 --topic user-event --from-beginning
```